# Table of Contents
    0. Preparing Data
    1. Cleanup
    2. Labels
    3. Analysis
    4. Model

In [140]:
#import necessary libraries
import findspark
findspark.init()
import math
import pandas as pd
import pyspark.sql
import pyspark.sql.functions as F
sc = SparkSession \
    .builder \
    .master("spark://master:7077") \
    .appName("EQ Work Sample Problems ") \
    .getOrCreate()

# 0. Preparing Data

In [135]:
df = pd.read_csv("Data/DataSample.csv" )

In [117]:
#see the data we're dealing with
df.head(3)

,_ID,TimeSt,Country,Province,City,Latitude,Longitude
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010


In [109]:
#see how many rows before sorting the data (so we know it works)
df.count()

_ID          22025
 TimeSt      22025
Country      22025
Province     22025
City         22025
Latitude     22025
Longitude    22025
dtype: int64

# 1. Cleanup

In [207]:
#change some of the column headings to be more user friendly
df = data_sample.rename(columns ={'_ID':'ID',' TimeSt':'TimeSt'})

In [209]:
#remove records with duplicate time stamps and geoinfo, not keeping the initial record
df.drop_duplicates(subset=["TimeSt","Latitude", "Longitude"], keep = False, inplace =True)

In [130]:
df.count()
#Now we can see 4052 suspicious logs were removed

ID           17973
TimeSt       17973
Country      17973
Province     17973
City         17973
Latitude     17973
Longitude    17973
dtype: int64

In [184]:
poi = pd.read_csv("data/POIList.csv")
df = data_sample.rename(columns ={' Latitude':'Latitude'})
poi.head()

,POIID,Latitude,Longitude
0,POI1,53.546167,-113.485734
1,POI2,53.546167,-113.485734
2,POI3,45.521629,-73.566024
3,POI4,45.224830,-63.232729


In [232]:
poi = poi.values.tolist()

# 2. Labels

### To solve this problem, I have broken it down into 3 steps:

 1. Find euclidean distance of requests from POI
 2. Get nearest POI
 3. Store the labels in order then append the list as a column to the DataFrame
    

In [244]:
#create a copy of the data samples dataframe but only with latitude and longitude columns, repeat for poi
cdf = df[["Latitude","Longitude"]]
#convert the copy into a list
cdf = cdf.values.tolist()

lst = [3,1,3,6]
al = [1,[5,34],3,4]
al[lst.index(min(lst))][0]

5

In [250]:

labels = []
for i in cdf:
    closest = []
    for x in poi:
        closest.append(math.dist(i, [x[1],x[2]]))
    #append the label of the POI to the labels list
    labels.append(poi[closest.index(min(closest))][0])
    
    
    

In [266]:
label = pd.Series(labels)
df['Label'] = label.values
df.head(2567)
#Labels seems to be working well

,ID,TimeSt,Country,Province,City,Latitude,Longitude,Label
0,4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123,POI3
1,4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090,POI3
2,4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010,POI3
3,4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730,POI3
4,4516613,2017-06-21 15:00:00.497,CA,ON,Stratford,43.37160,-80.97730,POI3
...,...,...,...,...,...,...,...,...
3060,4669263,2017-06-21 07:08:52.243,CA,ON,Wasaga Beach,44.52750,-80.00290,POI3
3061,4669285,2017-06-21 03:08:51.560,CA,ON,Brampton,43.69710,-79.79440,POI3
3062,4669293,2017-06-21 02:08:51.590,CA,NS,Sydney,46.16124,-60.17942,POI4
3063,4669321,2017-06-21 07:08:55.480,CA,ON,Kingston,44.26470,-76.55040,POI3


# 3. Analysis

    This problem will heavily depend on external libraries and functions

In [274]:
!python3 -m ipykernel install --user

Installed kernelspec python3 in C:\Users\tobio\AppData\Roaming\jupyter\kernels\python3
